# %% [markdown]
# # SUMO Traffic Light Simulation Analysis
# 
# ![Intersection](images/simulation.gif)

# %%

In [ ]:
import sys
sys.path.append('../scripts')
from data_loader import load_tripinfo_data
from visualization import plot_metrics, compare_scenarios
from metrics_calculator import calculate_metrics

import yaml
import pandas as pd
import matplotlib.pyplot as plt

# %%
# Load configuration
with open('../config/analysis_config.yaml') as f:
    config = yaml.safe_load(f)

scenarios = config['scenarios']
output_dir = Path(config['output_dir'])
images_dir = Path(config['images_dir'])

# %%
# Load and process all scenario data
scenario_dfs = {}
for name, path in scenarios.items():
    df = load_tripinfo_data(output_dir / path)
    scenario_dfs[name] = calculate_metrics(df)
    plot_metrics(df, name, images_dir)

# %%
# Generate comparison report
comparison_df = compare_scenarios(scenario_dfs, images_dir)
comparison_df.style.background_gradient(cmap='viridis')

# %%
# Advanced Analysis: Time-of-Day Patterns
hourly_metrics = []
for name, df in scenario_dfs.items():
    df['hour'] = df['depart'] // 3600
    hourly = df.groupby('hour').agg({
        'duration': 'mean',
        'waiting_time': 'mean',
        'speed': 'mean'
    })
    hourly['scenario'] = name
    hourly_metrics.append(hourly)

pd.concat(hourly_metrics).pivot_table(
    index='hour', 
    columns='scenario', 
    values=['duration', 'waiting_time']
).plot(subplots=True, figsize=(12, 8))
plt.savefig(images_dir / 'hourly_comparison.png')